In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

#setup
import os, sys
current_path = os.path.abspath('')
parent_path = os.path.dirname(current_path)
source_path = f'{parent_path}\\src'
if not source_path in sys.path: sys.path.append(source_path)

#customs
# from data_model import *
# from data_helper import *
from config import *
setup()

Config finished!


In [2]:
df = pd.read_csv('./data/V1.ExaltedOrb.csv', index_col='Date', parse_dates=True)
df.head()

,Value,League,Confidence,LeagueDay
Date,,,,
2016-09-08,64.00417,Essence,0,1.0
2016-09-09,64.51451,Essence,0,2.0
2016-09-10,65.00280,Essence,0,3.0
2016-09-11,65.05764,Essence,0,4.0
2016-09-12,65.98811,Essence,0,5.0


In [3]:
def train_val_split(df, selected_features):
    is_last_league = df['League'] == Constants.LEAGUES[-1]

    df_selected = df[selected_features]
    df_train = df_selected[~is_last_league]
    df_val = df_selected[is_last_league]
    return df_train, df_val
def print_train_val_stats(train,val):
    print('shapes: ', train.shape, val.shape)
    display(train.head())
    display(val.head())

In [4]:
df_train, df_val = train_val_split(df, ['Value','LeagueDay','Confidence'])
print_train_val_stats(df_train, df_val)

shapes:  (1719, 3) (88, 3)


,Value,LeagueDay,Confidence
Date,,,
2016-09-08,64.00417,1.0,0
2016-09-09,64.51451,2.0,0
2016-09-10,65.00280,3.0,0
2016-09-11,65.05764,4.0,0
2016-09-12,65.98811,5.0,0


,Value,LeagueDay,Confidence
Date,,,
2021-07-23,50.00000,1.0,1
2021-07-24,62.00000,2.0,0
2021-07-25,72.57994,3.0,0
2021-07-26,77.00000,4.0,0
2021-07-27,75.91450,5.0,0


In [84]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)
def make_multi_target(ts, steps, lead_time=1):
    return pd.concat(
        {
            f'y_step_{i}': ts.shift(-i)
            for i in range(lead_time, steps + lead_time)
        },
        axis=1)

from statsmodels.tsa.deterministic import DeterministicProcess

class FeatureEngineer:
    # def trend_features(self, df, training = True):
    #     if training:
    #         self.dp = DeterministicProcess(
    #             index = df.index,
    #             order = 4,
    #             drop = True
    #         )
    #         X = self.dp.in_sample().fillna(0)
    #     else:
    #         X = self.dp.out_of_sample(df.shape[0]).fillna(0)
        
    #     return X
    # def residual_features(self, df, training = True, lags = 4):
    #     if training:
    #         self.dp = DeterministicProcess(
    #             index = df.index,
    #             order = 1,
    #             drop = True
    #         )
    #         index_features = self.dp.in_sample().fillna(0)
    #     else:
    #         index_features = self.dp.out_of_sample(df.index).fillna(0)

    #     lags = make_lags(df['Value'], lags).fillna(0)
    #     selected_features = df[['Confidence', 'LeagueDay']]
    #     return pd.concat(
    #         [index_features, lags, selected_features],
    #         axis = 1
    #     )
    def target(self, df, steps = 5):
        return make_multi_target(df['Value'], steps).fillna(0)
    
    def time_features(self, df):
        df_return = df[['LeagueDay']]
        time_index = df.index


        df_return['day_of_week'] = time_index.dayofweek
        df_return['month'] = time_index.month

        return df_return

    def lag_features(self, df, lags = 4):
        return make_lags(df['Value'], lags).fillna(0)

    def rolling_features(self, df, local_window_size = 5, general_window_size=15):
        local_rolling = df['Value'].rolling(local_window_size, center = True)
        general_rolling = df['Value'].rolling(general_window_size, center = True)

        df_return = pd.DataFrame()
        df_return['rolling_mean'] = local_rolling.mean()
        df_return['rolling_min'] = local_rolling.min()
        df_return['rolling_max'] = local_rolling.max()
        df_return['general_rolling_mean'] = general_rolling.mean()
        df_return['general_rolling_min'] = general_rolling.min()
        df_return['general_rolling_max'] = general_rolling.max()
        return df_return        
    
    def expanded_features(self, df, min_period = 2):
        expanding = df['Value'].expanding(min_period, center = True)
        pd_return = pd.DataFrame()
        pd_return['expanding_mean'] = expanding.mean()
        pd_return['expanding_max'] = expanding.max()
        pd_return['expanding_min'] = expanding.min()
        return pd_return

    def all_features(self, df, fill0 = True):
        df_return =  pd.concat(
            [
                self.time_features(df),
                self.lag_features(df),
                self.rolling_features(df),
                self.expanded_features(df),
            ],
            
            axis = 1
        )
        return df_return.fillna(0) if fill0 else df_return

In [88]:
feature_engineer = FeatureEngineer()

X_train = feature_engineer.all_features(df_train)
y_train = feature_engineer.target(df_train)
X_val = feature_engineer.all_features(df_val)
y_val = feature_engineer.target(df_val)

F:\AppSSD\WorkTools\Anaconda\envs\gpu-ML\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\AppSSD\WorkTools\Anaconda\envs\gpu-ML\lib\site-packages\ipykernel_launcher.py:78: FutureWarning: The `center` argument on `expanding` will be removed in the future


In [89]:
for x in [X_train, X_val, y_train, y_val]:
    print(x.shape)
    display(x.head())

(1719, 16)


,LeagueDay,day_of_week,month,y_lag_1,y_lag_2,y_lag_3,y_lag_4,rolling_mean,rolling_min,rolling_max,general_rolling_mean,general_rolling_min,general_rolling_max,expanding_mean,expanding_max,expanding_min
Date,,,,,,,,,,,,,,,,
2016-09-08,1.0,3,9,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.00000,0.00000
2016-09-09,2.0,4,9,64.00417,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,64.259340,64.51451,64.00417
2016-09-10,3.0,5,9,64.51451,64.00417,0.00000,0.00000,64.913446,64.00417,65.98811,0.0,0.0,0.0,64.507160,65.00280,64.00417
2016-09-11,4.0,6,9,65.00280,64.51451,64.00417,0.00000,65.316060,64.51451,66.01724,0.0,0.0,0.0,64.644780,65.05764,64.00417
2016-09-12,5.0,0,9,65.05764,65.00280,64.51451,64.00417,66.230082,65.00280,69.08462,0.0,0.0,0.0,64.913446,65.98811,64.00417


(88, 16)


,LeagueDay,day_of_week,month,y_lag_1,y_lag_2,y_lag_3,y_lag_4,rolling_mean,rolling_min,rolling_max,general_rolling_mean,general_rolling_min,general_rolling_max,expanding_mean,expanding_max,expanding_min
Date,,,,,,,,,,,,,,,,
2021-07-23,1.0,4,7,0.00000,0.00000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0
2021-07-24,2.0,5,7,50.00000,0.00000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,56.000000,62.00000,50.0
2021-07-25,3.0,6,7,62.00000,50.00000,0.0,0.0,67.498888,50.00000,77.0,0.0,0.0,0.0,61.526647,72.57994,50.0
2021-07-26,4.0,0,7,72.57994,62.00000,50.0,0.0,72.498888,62.00000,77.0,0.0,0.0,0.0,65.394985,77.00000,50.0
2021-07-27,5.0,1,7,77.00000,72.57994,62.0,50.0,75.298888,72.57994,77.0,0.0,0.0,0.0,67.498888,77.00000,50.0


(1719, 5)


,y_step_1,y_step_2,y_step_3,y_step_4,y_step_5
Date,,,,,
2016-09-08,64.51451,65.00280,65.05764,65.98811,66.01724
2016-09-09,65.00280,65.05764,65.98811,66.01724,69.08462
2016-09-10,65.05764,65.98811,66.01724,69.08462,69.83986
2016-09-11,65.98811,66.01724,69.08462,69.83986,68.44797
2016-09-12,66.01724,69.08462,69.83986,68.44797,66.80162


(88, 5)


,y_step_1,y_step_2,y_step_3,y_step_4,y_step_5
Date,,,,,
2021-07-23,62.00000,72.57994,77.0000,75.91450,75.00000
2021-07-24,72.57994,77.00000,75.9145,75.00000,76.00000
2021-07-25,77.00000,75.91450,75.0000,76.00000,80.00000
2021-07-26,75.91450,75.00000,76.0000,80.00000,93.13224
2021-07-27,75.00000,76.00000,80.0000,93.13224,93.00000


In [91]:
out_names = ['X_train', 'X_val', 'y_train', 'y_val']
outs_dfs = [X_train, X_val, y_train, y_val]
for name, df in zip(out_names, outs_dfs):
    out_name = f'./data/{name}.csv'
    print(f'Saving {out_name}')
    df.reset_index().to_csv(out_name, index=False)


Saving ./data/X_train.csv
Saving ./data/X_val.csv
Saving ./data/y_train.csv
Saving ./data/y_val.csv
